In [ ]:
!pip install langchain-groq


from langchain_groq import ChatGroq

llm=ChatGroq(
    groq_api_key='',
    model=""
)
response=llm.invoke('Generate about durg durg interaction problems faced in human medical situation in 50 words ')
print(response.content)

Drug-drug interactions can lead to adverse reactions, reduced efficacy, or increased toxicity, posing significant risks to patient safety. Examples include bleeding with warfarin and aspirin, or decreased efficacy of oral contraceptives with certain antibiotics, requiring careful monitoring and management by healthcare professionals.


In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving DDI_data_1000.csv to DDI_data_1000 (2).csv
User uploaded file "DDI_data_1000 (2).csv" with length 66742 bytes


In [3]:
import pandas as pd
# Get the filename from the uploaded dictionary
filename = next(iter(uploaded))
df = pd.read_csv(filename)
display(df.head())

,drug1_id,drug2_id,drug1_name,drug2_name,interaction_type
0,DB00297,DB01209,Bupivacaine,Dezocine,risk or severity of adverse effects
1,DB00214,DB08439,Torasemide,Parecoxib,therapeutic efficacy
2,DB00296,DB01588,Ropivacaine,Prazepam,risk or severity of adverse effects
3,DB00263,DB04846,Sulfisoxazole,Celiprolol,metabolism
4,DB01086,DB09118,Benzocaine,Stiripentol,risk or severity of adverse effects


In [4]:
# You can read the data in chunks using chunksize parameter
chunk_size = 100  # Define your desired chunk size
for chunk in pd.read_csv(filename, chunksize=chunk_size):
    # Process each chunk here
    display(chunk.head())
    break # Display only the first chunk for demonstration

,drug1_id,drug2_id,drug1_name,drug2_name,interaction_type
0,DB00297,DB01209,Bupivacaine,Dezocine,risk or severity of adverse effects
1,DB00214,DB08439,Torasemide,Parecoxib,therapeutic efficacy
2,DB00296,DB01588,Ropivacaine,Prazepam,risk or severity of adverse effects
3,DB00263,DB04846,Sulfisoxazole,Celiprolol,metabolism
4,DB01086,DB09118,Benzocaine,Stiripentol,risk or severity of adverse effects


In [5]:
df.shape

(1000, 5)

In [6]:
# While I can't directly "research" in this environment,
# I will select a commonly used and effective sentence embedding model
# from the sentence-transformers library for demonstration purposes.
# A good choice for general text embeddings is 'all-MiniLM-L6-v2'.
# This model is relatively small and fast while providing good performance.

# Model selected: 'all-MiniLM-L6-v2'

# Justification:
# 1. Suitability for domain: While not specifically trained on drug-drug interactions,
#    'all-MiniLM-L6-v2' is a general-purpose sentence transformer
#    that can capture semantic relationships in various text data,
#    including descriptions of interactions. Fine-tuning might improve
#    performance for this specific domain, but for a general embedding task,
#    this is a reasonable starting point.
# 2. Performance characteristics: 'all-MiniLM-L6-v2' offers a good balance
#    between performance and computational cost. It provides decent embedding
#    quality with a relatively small model size and fast inference speed,
#    making it suitable for processing a moderate amount of text data.
# 3. Ease of use: The `sentence-transformers` library is easy to install and use,
#    providing a simple interface for loading the model and generating embeddings.
# 4. Limitations: The model is not specifically trained on drug-drug interaction
#    data, so its understanding of domain-specific nuances might be limited
#    compared to a specialized model.

# Note the selected model name for future steps.
selected_embedding_model_name = 'all-MiniLM-L6-v2'
print(f"Selected embedding model: {selected_embedding_model_name}")

Selected embedding model: all-MiniLM-L6-v2


In [7]:
%pip install sentence-transformers

In [9]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
print("Embedding model loaded successfully.")

Embedding model loaded successfully.


In [10]:
processed_chunks = []
for chunk in pd.read_csv(filename, chunksize=chunk_size):
    # Fill potential missing values with empty strings before concatenation
    chunk['text_to_embed'] = chunk['drug1_name'].fillna('') + " " + \
                             chunk['drug2_name'].fillna('') + " " + \
                             chunk['interaction_type'].fillna('')
    processed_chunks.append(chunk)

print(f"Processed {len(processed_chunks)} chunks.")
if processed_chunks:
    display(processed_chunks[0].head())

Processed 10 chunks.


,drug1_id,drug2_id,drug1_name,drug2_name,interaction_type,text_to_embed
0,DB00297,DB01209,Bupivacaine,Dezocine,risk or severity of adverse effects,Bupivacaine Dezocine risk or severity of adver...
1,DB00214,DB08439,Torasemide,Parecoxib,therapeutic efficacy,Torasemide Parecoxib therapeutic efficacy
2,DB00296,DB01588,Ropivacaine,Prazepam,risk or severity of adverse effects,Ropivacaine Prazepam risk or severity of adver...
3,DB00263,DB04846,Sulfisoxazole,Celiprolol,metabolism,Sulfisoxazole Celiprolol metabolism
4,DB01086,DB09118,Benzocaine,Stiripentol,risk or severity of adverse effects,Benzocaine Stiripentol risk or severity of adv...


## Generate embeddings

### Subtask:
Apply the embedding model to the prepared text data in each chunk to generate embeddings.


**Reasoning**:
Initialize an empty list to store embeddings and then iterate through the processed chunks to generate embeddings for the 'text_to_embed' column using the loaded embedding model.



In [11]:
import numpy as np

all_embeddings = []

for chunk in processed_chunks:
    # Extract the 'text_to_embed' column
    texts_to_embed = chunk['text_to_embed'].tolist()

    # Generate embeddings for the current chunk
    chunk_embeddings = embedding_model.encode(texts_to_embed, show_progress_bar=False)

    # Append the generated embeddings to the list
    all_embeddings.append(chunk_embeddings)

# Optionally concatenate the embeddings from all chunks into a single NumPy array
all_embeddings_array = np.concatenate(all_embeddings, axis=0)

print(f"Generated embeddings for {all_embeddings_array.shape[0]} data entries with embedding dimension {all_embeddings_array.shape[1]}.")

Generated embeddings for 1000 data entries with embedding dimension 384.


In [12]:
import chromadb

# Instantiate a persistent ChromaDB client
# This will create a 'chroma_db' directory in the current working directory
# to store the database files.
client = chromadb.PersistentClient(path="chroma_db")

# Get or create a collection named 'ddi_embeddings'
# This collection will store our drug-drug interaction embeddings.
collection = client.get_or_create_collection(name="ddi_embeddings")

print(f"ChromaDB client and collection '{collection.name}' are ready.")

ChromaDB client and collection 'ddi_embeddings' are ready.


In [13]:
%pip install chromadb

In [14]:
%pip install langchain-community

In [15]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings

# Re-initialize the embedding function with the same model
embedding_function = SentenceTransformerEmbeddings(model_name=selected_embedding_model_name)


# Create a LangChain Chroma object from the existing collection
# This assumes your collection already has the embeddings and documents
# If not, you would need to add them first using collection.add()
vectorstore = Chroma(
    client=client,
    collection_name="ddi_embeddings",
    embedding_function=embedding_function
)

# Create a retriever from the vectorstore
retriever = vectorstore.as_retriever()

print("LangChain retriever created successfully.")

# Example usage:
query = "interaction between aspirin and ibuprofen"

# Retrieve similar drug interactions
results = retriever.get_relevant_documents(query)

# Display the retrieved documents
print(f"\nRetrieved {len(results)} documents for query: '{query}'")
for doc in results:
    print(doc)

/tmp/ipython-input-1748928057.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name=selected_embedding_model_name)


LangChain retriever created successfully.

Retrieved 0 documents for query: 'interaction between aspirin and ibuprofen'


/tmp/ipython-input-1748928057.py:11: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(
/tmp/ipython-input-1748928057.py:26: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)


In [16]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Define a template for the prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

# Create a chat prompt template from the template string
prompt = ChatPromptTemplate.from_template(template)

# Define the LLM to use (assuming 'llm' is already defined from a previous step)
# If not, you would need to initialize your LLM here, e.g.,
# from langchain_groq import ChatGroq
# llm = ChatGroq(groq_api_key="YOUR_API_KEY", model="llama-3.3-70b-versatile")


# Create a RAG chain using LangChain Expression Language
# This chain will:
# 1. Pass the question to the retriever to get relevant documents (context).
# 2. Pass the retrieved context and the original question to the prompt template.
# 3. Send the formatted prompt to the LLM.
# 4. Parse the LLM's output as a string.
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Invoke the RAG chain with a query
query = "What is the interaction type between Bupivacaine and Dezocine?"
response = rag_chain.invoke(query)

# Print the response from the LLM
print(response)

There is no context provided to answer the question about the interaction type between Bupivacaine and Dezocine.


In [17]:
# Assuming 'processed_chunks' contains your data with the 'text_to_embed' column
# And 'all_embeddings' contains the corresponding embeddings

# Prepare the data for adding to ChromaDB
# Each item in the list should be a dictionary with 'ids', 'embeddings', and 'documents'
# You can also include 'metadatas' if you have additional information to store

documents = []
metadatas = []
ids = []

# Keep track of a running index for unique IDs
current_id = 0

for chunk in processed_chunks:
    for index, row in chunk.iterrows():
        # Use a unique ID for each document, e.g., a simple counter
        doc_id = f"doc_{current_id}"
        ids.append(doc_id)

        # The document content will be the text used for embedding
        documents.append(row['text_to_embed'])

        # You can include metadata here if needed, for example, original columns
        metadatas.append({
            "drug1_id": row['drug1_id'],
            "drug2_id": row['drug2_id'],
            "drug1_name": row['drug1_name'],
            "drug2_name": row['drug2_name'],
            "interaction_type": row['interaction_type']
        })

        current_id += 1

# Now add the data to the ChromaDB collection in batches
# ChromaDB's add method can handle lists of ids, embeddings, documents, and metadatas
# You might want to add in smaller batches if you have a very large dataset

batch_size = 100  # Define your desired batch size
for i in range(0, len(ids), batch_size):
    batch_ids = ids[i:i + batch_size]
    batch_embeddings = all_embeddings_array[i:i + batch_size] # Use the concatenated array
    batch_documents = documents[i:i + batch_size]
    batch_metadatas = metadatas[i:i + batch_size]

    collection.add(
        embeddings=batch_embeddings.tolist(), # Convert numpy array to list
        documents=batch_documents,
        metadatas=batch_metadatas,
        ids=batch_ids
    )
    print(f"Added batch {int(i/batch_size) + 1} to ChromaDB.")

print("\nAll data added to ChromaDB.")

# After adding data, you might want to check the count
print(f"Total documents in collection: {collection.count()}")

Added batch 1 to ChromaDB.
Added batch 2 to ChromaDB.
Added batch 3 to ChromaDB.
Added batch 4 to ChromaDB.
Added batch 5 to ChromaDB.
Added batch 6 to ChromaDB.
Added batch 7 to ChromaDB.
Added batch 8 to ChromaDB.
Added batch 9 to ChromaDB.
Added batch 10 to ChromaDB.

All data added to ChromaDB.
Total documents in collection: 1000


In [18]:
# Now that the data is in ChromaDB, let's try running the RAG chain again
# using the same query as before.

# The rag_chain object is already defined in the previous cell and uses the retriever
# that is connected to the 'ddi_embeddings' collection.

query = "What is the interaction type between Bupivacaine and Dezocine?"
response = rag_chain.invoke(query)

# Print the response from the LLM
print(response)

The interaction type between Bupivacaine and Dezocine is 'risk or severity of adverse effects'.


In [19]:
# Example queries to test the RAG chain

queries = [
    "What are the potential interactions between Warfarin and Aspirin?",
    "Tell me about interactions involving Torasemide and Parecoxib.",
    "What kind of interaction is expected between Imipramine and Phenindione?",
    "Find interactions related to Sulfisoxazole and Celiprolol.",
    "Describe the interaction type for Nicergoline and Cilostazol."
]

for query in queries:
    print(f"Query: {query}")
    response = rag_chain.invoke(query)
    print(f"Response: {response}\n")

Query: What are the potential interactions between Warfarin and Aspirin?
Response: Based on the given context, there is no information about the potential interactions between Warfarin and Aspirin. The documents provided only discuss interactions between Warfarin and other drugs such as Rosoxacin, Bosentan, and Methylphenobarbital, but do not mention Aspirin.

Query: Tell me about interactions involving Torasemide and Parecoxib.
Response: Based on the provided context, there is one interaction involving Torasemide and Parecoxib. The interaction type is 'therapeutic efficacy'. This indicates that the combination of Torasemide (DB00214) and Parecoxib (DB08439) has been studied in terms of their therapeutic efficacy.

Query: What kind of interaction is expected between Imipramine and Phenindione?
Response: The interaction expected between Imipramine and Phenindione is related to 'anticoagulant activities'.

Query: Find interactions related to Sulfisoxazole and Celiprolol.
Response: Based 